In [2]:
# Import necessary libraries
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained face detection model (Haar cascades)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load your trained emotion recognition model
# Load your trained emotion recognition model
emotion_model = load_model('model.h5')


# List of emotion categories (in the same order as used during training)
emotion_categories = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Perform emotion recognition on each detected face
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))  # Resize the face region to match model input size
        face_roi = face_roi.astype('float32') / 255.0  # Normalize pixel values
        face_roi = np.expand_dims(face_roi, axis=0)  # Add batch dimension
        emotion_predictions = emotion_model.predict(face_roi)

        predicted_emotion_index = np.argmax(emotion_predictions)
        predicted_emotion_label = emotion_categories[predicted_emotion_index]

        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display the predicted emotion label near the face
        cv2.putText(frame, predicted_emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display the updated frame
    cv2.imshow('Real-Time Emotion Detection', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 28ms/step
